<h2 align="center"> CSC 575 Final Project</h2>
<h2 align="center">NAWAAZ SHARIF</h2> 
<h2 align="center">MOHAMMED RASHIDUDDIN</h2> 
<h2 align="center">SYED NOOR RAZI ALI</h2> 


In [1]:
# Importing the reuqired packages and modules
import pandas as pd
import numpy as np
from numpy.linalg import norm
from IPython.display import Image
pd.set_option('display.max_colwidth', 50)
import itertools
import zipfile
import math
import xml.etree.ElementTree as ET
import urllib.request
import string
from nltk.corpus import stopwords
from nltk.stem import snowball
translator = str.maketrans('', '', string.punctuation)
stopwords = set(stopwords.words('english'))
from collections import defaultdict
import re
import gensim
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Fetching the GoogleNews-vectors-negative module for Word2vec to expand the query.
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(r'GoogleNews-vectors-negative300.bin', binary=True)


In [3]:
#Combining all the zip files to a single zip.

zipfile_names = ['ClinicalTrials.2021-04-27.part1.zip', 'ClinicalTrials.2021-04-27.part5.zip', 'ClinicalTrials.2021-04-27.part2.zip' , 'ClinicalTrials.2021-04-27.part3.zip' , 'ClinicalTrials.2021-04-27.part4.zip']

# Create a new zip file to store the combined contents
combined_zipfile = zipfile.ZipFile('Final_combined_ClinicalTrials.zip', 'w')

# Loop through each zip file and add its contents to the combined zip file
for filename in zipfile_names:
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        for member in zip_ref.infolist():
            combined_zipfile.writestr(member.filename, zip_ref.read(member.filename))

combined_zipfile.close()


In [4]:
%%time
# Reading the files from the XML to get the required ID and the context of the document
df_clinical_trial = pd.DataFrame(columns=['org_study_id', 'nct_id', 'brief_title', 'sponsor_agency', 'brief_summary' ,'detailed_description' , 'condition' , 'eligibility_condition' , 'eligibility_gender' , 'eligibility_minage' , 'eligibility_maxage'] , dtype=object)
with zipfile.ZipFile(r'Final_combined_ClinicalTrials.zip', 'r') as zip_ref:
    xml_files = [f for f in zip_ref.namelist() if f.endswith('.xml')]
    #considering only 1000 xml files.
    for xml_file in xml_files[0:10]:
        #extracting the zip
        zip_ref.extract(xml_file, path='temp')
        #parsing to read the contents
        tree = ET.parse(f'temp/{xml_file}')
        root = tree.getroot()
        #Orginization Study ID
        org_study_id = root.find('id_info/org_study_id').text if root.find('id_info/org_study_id') is not None else ''
        #Document ID
        nct_id = root.find('id_info/nct_id').text.replace('NCT00', 'D').strip() if root.find('id_info/nct_id') is not None else ''
        #Brief Title of the document
        brief_title = root.find('brief_title').text if root.find('brief_title') is not None else ''
        sponsor_agency = root.find('sponsors/lead_sponsor/agency').text if root.find('sponsors/lead_sponsor/agency') is not None else ''
        #Brief Summary of Clinical Trials
        brief_summary = root.find('brief_summary/textblock').text.replace('\n', ' ').replace('\r', ' ').strip() if root.find('brief_summary/textblock') is not None else ''
        #Detailed Description of Clinical Trial
        detailed_description = root.find('detailed_description/textblock').text.replace('\n', ' ').replace('\r', ' ').strip() if root.find('detailed_description/textblock')is not None else ''
        condition = root.find('condition').text if root.find('condition') is not None else ''
        eligibility_condition = root.find('eligibility/criteria/textblock').text.replace('\n', ' ').replace('\r', ' ').strip() if root.find('eligibility/criteria/textblock') is not None else ''
        eligibility_gender = root.find('eligibility/gender').text if root.find('eligibility/gender') is not None else ''
        eligibility_minage = root.find('eligibility/minimum_age').text if root.find('eligibility/minimum_age') is not None else ''
        eligibility_maxage = root.find('eligibility/maximum_age').text if root.find('eligibility/maximum_age') is not None else ''
        df_clinical_trial.loc[len(df_clinical_trial.index)] =[org_study_id, nct_id, brief_title, sponsor_agency, brief_summary ,detailed_description , condition , eligibility_condition , eligibility_gender , eligibility_minage , eligibility_maxage]
# print(df_clinical_trial)
    

Wall time: 4.7 s


In [5]:
df_clinical_trial.head()

,org_study_id,nct_id,brief_title,sponsor_agency,brief_summary,detailed_description,condition,eligibility_condition,eligibility_gender,eligibility_minage,eligibility_maxage
0,NCRR-M01RR01070-0506,D000102,Congenital Adrenal Hyperplasia: Calcium Channe...,National Center for Research Resources (NCRR),This study will test the ability of extended r...,This protocol is designed to assess both acute...,Congenital Adrenal Hyperplasia,Inclusion Criteria: - diagnosed ...,All,14 Years,35 Years
1,NCRR-M01RR00400-0587,D000104,Does Lead Burden Alter Neuropsychological Deve...,National Center for Research Resources (NCRR),Inner city children are at an increased risk f...,,Lead Poisoning,Inclusion Criteria: - Pregnant m...,Female,N/A,N/A
2,2002LS032,D000105,Vaccination With Tetanus and KLH to Assess Imm...,"Masonic Cancer Center, University of Minnesota",The purpose of this study is to learn how the ...,Patients will receive each vaccine once only c...,Cancer,Inclusion Criteria: - Patients m...,All,18 Years,N/A
3,NCRR-M01RR03186-9943,D000106,41.8 Degree Centigrade Whole Body Hyperthermia...,National Center for Research Resources (NCRR),Recently a non-toxic system for whole body hyp...,,Rheumatic Diseases,Inclusion Criteria: - Patients a...,All,18 Years,65 Years
4,NCRR-M01RR00109-0737,D000107,Body Water Content in Cyanotic Congenital Hear...,National Center for Research Resources (NCRR),Adults with cyanotic congenital heart disease ...,,"Heart Defects, Congenital",Inclusion Criteria: - Resting bl...,All,17 Years,60 Years


In [6]:
df_clinical_trial.shape

(10, 11)

In [7]:
# Converting the dataframe to HashMap/dictionary where the nct_id our document id is the key and the rest columns are values
df_clinical_trialdict = df_clinical_trial.set_index('nct_id').apply(lambda x: x['detailed_description']+ '_' + x['brief_title']+ '_' + x['brief_summary']+ '_' + x['eligibility_condition']+ '_' + x['condition'], axis=1).to_dict()
df_clinical_trialdict

{'D000102': 'This protocol is designed to assess both acute and chronic effects of the calcium channel        antagonist, nifedipine, on the hypothalamic-pituitary-adrenal axis in patients with        congenital adrenal hyperplasia. The multicenter trial is composed of two phases and will        involve a double-blind, placebo-controlled parallel design. The goal of Phase I is to examine        the ability of nifedipine vs. placebo to decrease adrenocorticotropic hormone (ACTH) levels,        as well as to begin to assess the dose-dependency of nifedipine effects. The goal of Phase II        is to evaluate the long-term effects of nifedipine; that is, can attenuation of ACTH release        by nifedipine permit a decrease in the dosage of glucocorticoid needed to suppress the HPA        axis? Such a decrease would, in turn, reduce the deleterious effects of glucocorticoid        treatment in CAH._Congenital Adrenal Hyperplasia: Calcium Channels as Therapeutic Targets_This study will tes

In [8]:
#Fetching the data from the XML file and converting ino DF.
def queries_topics():
    queries=pd.DataFrame(columns=['Queries'])
    url = 'https://www.trec-cds.org/topics2021.xml'

    # headers = {'User-Agent': 'Mozilla/5.0' , "Accept": "application/xml"}
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',
    }
    req = urllib.request.Request(url, headers=headers)
    xml_data = urllib.request.urlopen(req).read()


    root = ET.fromstring(xml_data)
    for topic in root.iter('topic'):
        queries.loc[len(queries.index)] =[topic.text.strip()]
    return queries


In [9]:
# Checking our Queries
queries_topics()

,Queries
0,Patient is a 45-year-old man with a history of...
1,"48 M with a h/o HTN hyperlipidemia, bicuspid a..."
2,A 32 yo woman who presents following a severe ...
3,"This is a 44 year old female with PMH of PCOS,..."
4,"74M hx of CAD s/p CABG, EF 60% prior CVA (no r..."
...,...
70,The patient is a 34-year-old obese woman who c...
71,The patient is a 16-year-old girl recently dia...
72,The patient is a 3-day-old female infant with ...
73,The patient is a 53-year-old man complaining o...


In [10]:
# Selecting a random query to do a search engine retrieval.
q=queries_topics()['Queries'].loc[1]


In [11]:
# Creating a function to remove punctuations, stopwords and stem words
def preprocessing_documents():
    df_clinical_dict ={}
    # loop through the dictionary and process the values
    for key, value in df_clinical_trialdict.items():
        # Removing numbers, symbols and punctuations
        value=re.sub(r'≥|µ|\d+', '', value).translate(translator).lower().split()
        # Remove stopwords from the hashmap D
        filtered_words = [word for word in value if word not in stopwords and len(word) > 1]
        # Creating a stem of all the words from the document
        stemming = snowball.SnowballStemmer('english')
        stemming_wwrds = [stemming.stem(x) for x in filtered_words]
        df_clinical_dict[key] = list(stemming_wwrds)
    #Final pre-processed document
    return df_clinical_dict

In [12]:
# creating a second dictionary to store the token and the list of documents it is present and the number of occurences
def doc_occurences():
    second_dict={}
    for key,value in  preprocessing_documents().items():
    #     key=int(key)
        for i in range(len(value)): # running through each key and its respective value
            if value[i] in second_dict:  # checking whether the value is present or not 
                if key not in second_dict[value[i]].keys(): 
    # this condition is because when the loop goes to second document id it has to add multiple occurences of a particular word
                    second_dict[value[i]][key]=1
                else:
                    second_dict[value[i]][key]+=1 # else increase the count by 1
            else:           
                second_dict[value[i]]={key:1} # create a multi level dictionary to add the words , the document id and the ouccrence
    return second_dict

In [13]:
# Sorting the dictionary keyas based on Alphabetic order 
sorted_dict = {key: value for key, value in sorted(doc_occurences().items())}


In [14]:
# Displaying it in a table format for the sorted dictionary whihc contains the unique word, Total Frequency and The Document Frequency.
print('Word'+'\t'+'Total Freq'+'\t'+'Document Freq'+'\t'+'Posting(Doc-ID, Count)')

for key, value in sorted_dict.items():
    column1=key
    column2 = sum(value.values())
    column3 = len(value)
    column4 = list(value.items())

    print(f"{ column1:<15}{column2:<15}{column3:<15}{column4}")


Word	Total Freq	Document Freq	Posting(Doc-ID, Count)
abil           3              2              [('D000102', 2), ('D000105', 1)]
acanthosi      1              1              [('D000112', 1)]
accommod       1              1              [('D000113', 1)]
accord         2              2              [('D000106', 1), ('D000113', 1)]
acr            1              1              [('D000106', 1)]
acrfor         1              1              [('D000106', 1)]
acth           2              1              [('D000102', 2)]
activ          4              3              [('D000106', 2), ('D000107', 1), ('D000108', 1)]
acuiti         1              1              [('D000113', 1)]
acut           2              2              [('D000102', 1), ('D000105', 1)]
add            1              1              [('D000108', 1)]
addit          7              2              [('D000111', 2), ('D000113', 5)]
adequ          3              1              [('D000106', 3)]
adher          1              1              

In [15]:
### Weighted Index

In [16]:
# Creating a idf dictionary to get the words and the idf values for all the words in the document
idf_dict={}
N = df_clinical_trial.shape[0]

def compute_tfidf(term, doc_id):
    tf = sorted_dict[term][doc_id]
    idf = math.log2(N / len(sorted_dict[term]))
    idf_dict[term]=math.log2(N / len(sorted_dict[term]))
    return tf * idf

weighted_index = {}
for term in sorted_dict:
    weighted_index[term] = {}
    for doc_id in sorted_dict[term]:
        weighted_index[term][doc_id] = compute_tfidf(term, doc_id)


In [17]:
#Here we have idf values for our dataset 
idf_dict

{'abil': 2.321928094887362,
 'acanthosi': 3.321928094887362,
 'accommod': 3.321928094887362,
 'accord': 2.321928094887362,
 'acr': 3.321928094887362,
 'acrfor': 3.321928094887362,
 'acth': 3.321928094887362,
 'activ': 1.736965594166206,
 'acuiti': 3.321928094887362,
 'acut': 2.321928094887362,
 'add': 3.321928094887362,
 'addit': 2.321928094887362,
 'adequ': 3.321928094887362,
 'adher': 3.321928094887362,
 'adipos': 3.321928094887362,
 'adjuv': 3.321928094887362,
 'administ': 3.321928094887362,
 'adren': 3.321928094887362,
 'adrenocorticotrop': 3.321928094887362,
 'adult': 3.321928094887362,
 'affect': 3.321928094887362,
 'age': 1.3219280948873624,
 'agespecif': 3.321928094887362,
 'aim': 3.321928094887362,
 'allergi': 3.321928094887362,
 'allow': 1.736965594166206,
 'along': 2.321928094887362,
 'alreadi': 3.321928094887362,
 'also': 2.321928094887362,
 'alter': 3.321928094887362,
 'although': 3.321928094887362,
 'american': 3.321928094887362,
 'among': 3.321928094887362,
 'anecdot': 3

In [18]:
def word2vec_expand(query, num_words=5, threshold=0.5):
    # Finding the top 5 similar words for the given query 
    # Here we are wxpanding the query based on Synonymns and the Goooglenews we have imported before.
    tokens = query.lower().split()

    # Find most similar words using Word2Vec
    similar_words = []
    for token in tokens:
        try:
            sim_words = word2vec_model.similar_by_word(token, topn=num_words)
            similar_words.extend([w[0] for w in sim_words if w[1] > threshold])
        except KeyError:
            pass

    # Combine original query with similar words
    expanded_query = ' '.join(tokens + similar_words)
    # This gives us the most similar words.
    return expanded_query


In [19]:
# This function does the pre-processing of the data where it removeds punctuations, stop words and does stemming
def userquery(query):
    query_idf={}
    query=re.sub(r'≥|µ|\d+', '', query).translate(translator).lower().split()

    filtered_words = [word for word in query if word not in stopwords and len(word) > 1]

    
    stemming = snowball.SnowballStemmer('english')
    stemming_wwrds = [stemming.stem(x) for x in filtered_words]
    for i in stemming_wwrds:
        if i in idf_dict.keys():
            query_idf[i]=idf_dict[i]
        else:
             pass
    # Final Query DataFrame having query words and its respective IDF values.
    return query_idf

In [20]:
# Checking our imlementaiton of expanded and user query function
expandedquery=word2vec_expand(q, num_words=5, threshold=0.5)
q=userquery(expandedquery)
# Here we have the vector representation of the query
q

{'present': 2.321928094887362,
 'progress': 3.321928094887362,
 'sever': 2.321928094887362,
 'function': 1.736965594166206,
 'pressur': 1.736965594166206,
 'clinic': 1.736965594166206,
 'toler': 3.321928094887362,
 'evalu': 1.736965594166206,
 'echo': 3.321928094887362,
 'confirm': 3.321928094887362,
 'critic': 3.321928094887362,
 'show': 3.321928094887362,
 'dilat': 3.321928094887362,
 'under': 3.321928094887362,
 'patient': 1.3219280948873624,
 'replac': 2.321928094887362,
 'scan': 3.321928094887362,
 'us': 2.321928094887362,
 'also': 2.321928094887362,
 'cardiac': 3.321928094887362,
 'heart': 1.736965594166206,
 'diseas': 1.0,
 'serious': 3.321928094887362,
 'alreadi': 3.321928094887362,
 'prior': 3.321928094887362,
 'anoth': 3.321928094887362,
 'last': 3.321928094887362,
 'import': 2.321928094887362,
 'shown': 3.321928094887362,
 'leav': 3.321928094887362,
 'respiratori': 3.321928094887362,
 'addit': 2.321928094887362,
 'howev': 3.321928094887362,
 'cardiovascular': 2.3219280948873

In [21]:
# This function is to calculate the document length for its repsective documents and document ids for all similairites.
def doclength(similarity):
    doc_length={}
    #Document Lenght for Cosine Similarity
    if similarity.lower()=='cosine':
        for key, value in sorted_dict.items():
            for document in value.keys():
                if document not in doc_length:
                    doc_length[document]=(sorted_dict[key][document]*idf_dict[key])**2
                else:
                    doc_length[document]+=(sorted_dict[key][document]*idf_dict[key])**2
        for key in doc_length:
            doc_length[key] = math.sqrt(doc_length[key])
    else:
        #Document Length for Jaccard and Dice Similarity 
        for key, value in sorted_dict.items():
            for document in value.keys():
                if document not in doc_length:
                    doc_length[document]=(sorted_dict[key][document]*idf_dict[key])**2
                else:
                    doc_length[document]+=(sorted_dict[key][document]*idf_dict[key])**2
        for key in doc_length:
            doc_length[key] = np.sum(doc_length[key])
        
    return doc_length


In [22]:
# queries length for its respective similarity

def querieslength(q,similarity):
    sum_queries_length=0
    # Cosine
    if similarity.lower()=='cosine':
        for key in q:
            sum_queries_length+= (idf_dict[key])**2
        sum_queries_length=math.sqrt(sum_queries_length)
    # Jaccard or Dice
    elif similarity.lower()=='jaccard' or similarity.lower()=='dice':
        for key in q:
            sum_queries_length+= (idf_dict[key])**2
        sum_queries_length=np.sum(sum_queries_length)      
    # This gives us the square root of sum of squares of idf values for cosine and np.sum for Jaccard or Dice
    return sum_queries_length


In [23]:
#This function runs iteratively for all the different similarities and gives us the final df with all the similarities 
#of the query and the documents.
def similar_documents(q):
    similarities=["Cosine" , 'Jaccard' , 'Dice']
    diff_similarity_df = pd.DataFrame(columns=["Cosine" , 'Jaccard' , 'Dice'])
    dictionaries_list=[]
    for i in similarities:
        document_ids= list(df_clinical_trial['nct_id'])
        document_dict = {document: 0 for document in document_ids}

        for term in q:
    #run for all the terms in the query
            for key in sorted_dict[term].keys():
            #Numerator
                document_dict[key]+=(sorted_dict[term][key]*idf_dict[term])*(1*idf_dict[term])
        for key in document_dict:
            if key in doclength(i):
                #Cosine Similairty of Queries and Documents
                if i.lower()=='cosine':
                    document_dict[key] = document_dict[key]/(querieslength(q,i)*doclength(i)[key])

                elif i.lower()=='dice':
                     #Dice Similairty of Queries and Documents
                    document_dict[key] = 2*document_dict[key]/(querieslength(q,i)*doclength(i)[key])
                else:
                     #Jaccard Similairty of Queries and Documents
                    document_dict[key] = document_dict[key]/((querieslength(q,i)*doclength(i)[key])-document_dict[key])
        dictionaries_list.append(document_dict)

    diff_similarity_df = pd.DataFrame.from_dict(dictionaries_list )
    diff_similarity_df=diff_similarity_df.T
    diff_similarity_df.columns = similarities
    return diff_similarity_df


In [24]:
# This function is just to get the brief title from the original df and the respective similairyt with document ID.
def documents_contents(similarity,q):
    diff_1 = similar_documents(q)
    diff_1 = diff_1[(diff_1 != 0).any(axis=1)].reset_index()
    diff_2 = df_clinical_trial
    df_doc_contents = pd.merge(diff_1 , diff_2 , left_on='index' , right_on='nct_id')
    df_doc_contents=df_doc_contents.rename(columns={'index':'Document ID'})
    df_doc_contents=df_doc_contents[['Document ID' , 'Cosine' , 'Jaccard','Dice' , 'brief_title']]
    similarity=similarity.capitalize()
    df_doc_contents=df_doc_contents.sort_values(by=similarity ,ascending=False)[['Document ID' ,similarity, 'brief_title']].reset_index(drop=True)
    return df_doc_contents
    

In [25]:
# Top 10 Cosine Similar Documents
documents_contents('COSINE',q).iloc[:10]

,Document ID,Cosine,brief_title
0,D000106,0.144860,41.8 Degree Centigrade Whole Body Hyperthermia...
1,D000113,0.130760,Correction of Myopia Evaluation Trial (COMET)
2,D000111,0.074940,Intraoral Grafting of Ex Vivo Produced Oral Mu...
3,D000104,0.052333,Does Lead Burden Alter Neuropsychological Deve...
4,D000108,0.052024,Effects of Training Intensity on the CHD Risk ...
5,D000112,0.046971,Prevalence of Carbohydrate Intolerance in Lean...
6,D000102,0.046934,Congenital Adrenal Hyperplasia: Calcium Channe...
7,D000107,0.046167,Body Water Content in Cyanotic Congenital Hear...
8,D000105,0.036725,Vaccination With Tetanus and KLH to Assess Imm...
9,D000110,0.016063,Influence of Diet and Endurance Running on Int...


In [26]:
# Top 10 Jaccard Similarity Documents
documents_contents('Jaccard',q).iloc[:10]

,Document ID,Jaccard,brief_title
0,D000106,0.000165,41.8 Degree Centigrade Whole Body Hyperthermia...
1,D000112,0.000100,Prevalence of Carbohydrate Intolerance in Lean...
2,D000107,0.000084,Body Water Content in Cyanotic Congenital Hear...
3,D000108,0.000083,Effects of Training Intensity on the CHD Risk ...
4,D000111,0.000080,Intraoral Grafting of Ex Vivo Produced Oral Mu...
5,D000113,0.000076,Correction of Myopia Evaluation Trial (COMET)
6,D000104,0.000075,Does Lead Burden Alter Neuropsychological Deve...
7,D000102,0.000061,Congenital Adrenal Hyperplasia: Calcium Channe...
8,D000105,0.000021,Vaccination With Tetanus and KLH to Assess Imm...
9,D000110,0.000019,Influence of Diet and Endurance Running on Int...


In [27]:
# Top 10 Dice Similarity Documents
documents_contents('DICE',q).iloc[:10]

,Document ID,Dice,brief_title
0,D000106,0.000329,41.8 Degree Centigrade Whole Body Hyperthermia...
1,D000112,0.000200,Prevalence of Carbohydrate Intolerance in Lean...
2,D000107,0.000167,Body Water Content in Cyanotic Congenital Hear...
3,D000108,0.000166,Effects of Training Intensity on the CHD Risk ...
4,D000111,0.000161,Intraoral Grafting of Ex Vivo Produced Oral Mu...
5,D000113,0.000152,Correction of Myopia Evaluation Trial (COMET)
6,D000104,0.000150,Does Lead Burden Alter Neuropsychological Deve...
7,D000102,0.000121,Congenital Adrenal Hyperplasia: Calcium Channe...
8,D000105,0.000042,Vaccination With Tetanus and KLH to Assess Imm...
9,D000110,0.000037,Influence of Diet and Endurance Running on Int...


In [28]:
# Comverting the Query to a dataframe where rest of the words are filled with zero except query terms
def query_df(q):
    df_query1 = pd.DataFrame(columns=list(sorted_dict.keys()))

    # Populate the DataFrame with the values from the given dictionary
    for key, value in q.items():
        if key in df_query1.columns:
            df_query1.loc[0, key] = value

    # Fill missing values with zeroes
    df_query1 = df_query1.fillna(0)
    return df_query1

In [29]:
# Converting the sorted dictionary to a pandas df
def rawtm_df():
    df_rawterms = pd.DataFrame.from_dict(sorted_dict, orient='index')
    df_rawterms=df_rawterms.fillna(0)
    df_rawterms=df_rawterms.T
    df_rawterms=df_rawterms.reindex(sorted(df_rawterms.columns), axis=1)
    return df_rawterms

In [30]:
# using relevance judments we are expanding the query.
def modify_query(Q, R, NR, alpha = 0.5, beta= 0.25) :
    # getting the Q document
    df_rawterms=rawtm_df()
    query_docs=rawtm_df().copy()
    query_words=query_docs.copy()
    new_row = pd.Series(Q, name='Q' , index=query_words.columns)
    query_words = query_words.append(new_row)
    # getting the relevant documents
    for i in R:
        new_row = pd.Series(df_rawterms.loc[i], name='R' , index=query_words.columns)
        query_words = query_words.append(new_row)
    relevance_docs=query_words.loc['R']
    average_relavance=relevance_docs.mean()
    # getting the non relevant documents.
    for i in NR:
        new_row = pd.Series(df_rawterms.loc[i], name='NR' , index=query_words.columns)
        query_words = query_words.append(new_row)
    nonrelevance_docs=query_words.loc['NR']
    average_nonrelavance=nonrelevance_docs.mean()
    Q1 = query_words.loc['Q'] + (alpha *average_relavance ) - (beta *average_nonrelavance)
    queries_tm = pd.DataFrame()

    queries_tm['Q'] =  query_words.loc['Q']
    queries_tm['Q1'] =  Q1
    queries_tm=queries_tm.T
    queries_tm.iloc[queries_tm < 0] = 0
    return queries_tm

In [32]:
#Check the new query for all the Relevant and Non-Relvant documents
Q=list(query_df(q).iloc[0])
R = ['D000106','D000113','D000102']
NR =['D000111','D000104','D000108']
alpha = 0.5
beta = 0.25
query_result = modify_query(Q, R, NR, alpha, beta)
query_result

,abil,acanthosi,accommod,accord,acr,acrfor,acth,activ,acuiti,acut,...,within,without,women,womenth,work,would,xl,year,yet,york
Q,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
Q1,0.333333,0.0,0.166667,0.333333,0.166667,0.166667,0.333333,0.25,0.166667,0.166667,...,0.0,0.083333,0.0,0.0,0.0,0.166667,0.166667,0.5,0.0,0.166667


In [33]:
# Get the difference between rows 1 and 2
query_resultdiff = query_result.iloc[1] - query_result.iloc[0]

# Create a mask for positive and negative values
positive_mask = query_resultdiff > 0
negative_mask = query_resultdiff < 0
                                                                
# Get the columns where values have increased and decreased
increased_cols = query_resultdiff[positive_mask].index
decreased_cols = query_resultdiff[negative_mask].index

print("Increased columns:", increased_cols)
print("Decreased columns:", decreased_cols)


Increased columns: Index(['abil', 'accommod', 'accord', 'acr', 'acrfor', 'acth', 'activ',
       'acuiti', 'acut', 'addit',
       ...
       'well', 'westergren', 'whether', 'whole', 'wisconsin', 'without',
       'would', 'xl', 'year', 'york'],
      dtype='object', length=393)
Decreased columns: Index(['alreadi', 'anoth', 'howev', 'leav', 'replac', 'under', 'us'], dtype='object')


In [34]:
# Checking the Cosine similairity of Documnets and the two Queries (old and new)
from sklearn.metrics.pairwise import cosine_similarity
Q_cosinesimilarity = cosine_similarity(query_result.loc['Q'].to_frame().T , rawtm_df()).flatten()
Q1_cosinesimilarity = cosine_similarity(query_result.loc['Q1'].to_frame().T , rawtm_df()).flatten()
data = {'Q': Q_cosinesimilarity, 'Q1': Q1_cosinesimilarity}
sim_documents = pd.DataFrame(data)
sim_documents=sim_documents.set_index(rawtm_df().index)
sim_documents=sim_documents.T

In [35]:
# Cosine similarity of document and the query.
sim_documents

,D000102,D000105,D000112,D000113,D000106,D000107,D000108,D000111,D000104,D000110
Q,0.067448,0.050727,0.036615,0.138264,0.147527,0.062618,0.064020,0.084142,0.045236,0.021961
Q1,0.232333,0.104838,0.074813,0.428722,0.324805,0.091931,0.089033,0.105036,0.088853,0.053861


In [36]:
query_resultdiff = query_result.iloc[1] - query_result.iloc[0]

# Create a mask for positive and negative values
positive_mask = query_resultdiff > 0
negative_mask = query_resultdiff < 0
                                                                
# Get the columns where values have increased and decreased
increased_cols = query_resultdiff[positive_mask].index
decreased_cols = query_resultdiff[negative_mask].index

print("Increased columns:", increased_cols)
print("Decreased columns:", decreased_cols)


Increased columns: Index(['abil', 'accommod', 'accord', 'acr', 'acrfor', 'acth', 'activ',
       'acuiti', 'acut', 'addit',
       ...
       'well', 'westergren', 'whether', 'whole', 'wisconsin', 'without',
       'would', 'xl', 'year', 'york'],
      dtype='object', length=393)
Decreased columns: Index(['alreadi', 'anoth', 'howev', 'leav', 'replac', 'under', 'us'], dtype='object')


## GUI Application for Search Retrieval System

In [37]:
# Importing Respective Modules to run our GUI.
import tkinter as tk
import customtkinter
from tabulate import tabulate
customtkinter.set_appearance_mode("dark") 
customtkinter.set_default_color_theme("dark-blue")  
root = customtkinter.CTk()
root.geometry("1024x600")
# TextBox to get our Input Query
entry = customtkinter.CTkTextbox(master=root, width=400,height=200 )
entry.pack()

textbox = customtkinter.CTkTextbox(master=root , width=350)
textbox.pack()
# Creating a label for the text box of input query.
label = customtkinter.CTkLabel(master=root, text="Enter your Input Query in the Textbox")
label.place(relx=0.08, rely=0.1)
label.configure(text_color='orange')
# Creating a label for Relevant Documents
label1 = customtkinter.CTkLabel(master=root, text="Relevant Documents")
label1.place(x=780,y=2)
label1.configure(text_color='orange')
# Creating an Entry box for Relevant documents
entry2=customtkinter.CTkEntry(master=root, width=300,placeholder_text="Enter your relevant docs seperated by comma")
entry2.pack()
entry2.place(x=720,y=25)
# Creating an Label for Non-Relevant Documents
label2 = customtkinter.CTkLabel(master=root, text="Non-Relevant Documents")
label2.place(x=780,y=60)
label2.configure(text_color='orange')
# Creating an Entry box for Non-Relevant documents
entry3=customtkinter.CTkEntry(master=root, width=300,placeholder_text="Enter your non-rel docs seperated by comma")
entry3.pack()
entry3.place(x=720,y=85)

# Creating a label for our random queries generated.

label4 = customtkinter.CTkLabel(master=root, text="Random Sample Query to give in Input Query.")
label4.place(x=20,y=350)
label4.configure(text_color='orange')
# Converting a df row to a string to show the sample query.
table = str((queries_topics().sample(n=1).to_string(index=False))).replace('\n', ' ').replace('\t', ' ').replace('-',"").replace('Queries',"").strip()
textbox.configure(text_color='white')
textbox.insert("2.0", table)
textbox.pack(padx=20, pady=250)
textbox.place(x=10,y=380)
 # THe output of the modified query
textbox2 = customtkinter.CTkTextbox(master=root , width=250)
textbox2.configure(fg_color='transparent' , text_color='white')
textbox2.pack()

def mydelete():
    # Clear the outputs from all the generated outputs
    global output_label, cosine_label, jaccard_label, dice_label
    if output_label is not None:
        output_label.destroy()
        output_label = None
    if cosine_label is not None:
        cosine_label.destroy()
        cosine_label = None
    if jaccard_label is not None:
        jaccard_label.destroy()
        jaccard_label = None
    if dice_label is not None:
        dice_label.destroy()
        dice_label = None

def query_modifier():
    # This function generated the modifed query having necessary terms to modify the query
    # This also takes the inut from the user for Releavnt and Non-Relevant terms
    R=entry2.get().split(',')
    NR = entry3.get().split(',')
    Q=tk_query
    newquery_result = modify_query(Q, R, NR, alpha = 0.5, beta= 0.25)
    query_resultdiff = newquery_result.iloc[1] - newquery_result.iloc[0]
    positive_mask = query_resultdiff > 0
    increased_cols = query_resultdiff[positive_mask].index
#     print("Increased columns:", increased_cols)
    textbox2.insert("0.0", f'After Query Expansion using Rocchio Method you can modify your vocabulary using these words {increased_cols.tolist()}' )
   
    textbox2.place(x=720,y=150)
#     query_modifier_msg = tk.Message(master=root , text=increased_cols  )
#     query_modifier_msg.pack(padx=20, pady=20)
#     query_modifier_msg.place(x=850,y=200 )
        
def myclick():
    # this funcaiton gets the query from the user and gives us the similar documents for all similarities
    global output_label
    global tk_query
    value1=entry.get("0.0", "end")
    expanded_query=word2vec_expand(value1, num_words=5, threshold=0.5)
    tk_query=userquery(expanded_query)
    output_label = tk.Message(master=root , text=similar_documents(tk_query).iloc[:10].to_string())
    output_label.pack(padx=10, pady=10)
    output_label.place(x=550,y=400 )

def topcosine():
    # Gives us the Cosine similar documents
    global cosine_label
    cosine_label = tk.Message(master=root , text=documents_contents('COSINE',tk_query).iloc[:10].to_string() ,  width=1700)
    cosine_label.pack(padx=20, pady=20)
    cosine_label.place(x=450,y=400 )

def topjaccard():
    # Gives us the Jaccard similar documents
    global jaccard_label
    jaccard_label = tk.Message(master=root , text=documents_contents('JACCARD',tk_query).iloc[:10].to_string() ,  width=1700)
    jaccard_label.pack(padx=20, pady=20)
    jaccard_label.place(x=450,y=400 )

def topdice():
    ## Gives us the Dice similar documents
    global dice_label
    dice_label = tk.Message(master=root , text=documents_contents('DICE',tk_query).iloc[:10].to_string() ,  width=1700)
    dice_label.pack(padx=20, pady=20)
    dice_label.place(x=450,y=400 )


#Button for Top Documents for all Similairties
button = customtkinter.CTkButton(master=root, text="Get the top documents for all similarities", command=myclick)
button.pack()
button.place(x=25, y=230)
#Delete Button to clear the output.
delbutton = customtkinter.CTkButton(master=root, text="Clear the Output", command=lambda: [mydelete()] ,fg_color="red")
delbutton.pack()
delbutton.place(x=390, y=205)
#Cosine Button to get the top 10 similar documents .
cosinebutton = customtkinter.CTkButton(master=root, text="Get the cosine values", command=topcosine)
cosinebutton.pack()
cosinebutton.place(x=25, y=260)
#Jaccard Button to get the top 10 similar documents .
jacdelbutton = customtkinter.CTkButton(master=root, text="Get the Jaccard values", command=topjaccard)
jacdelbutton.pack()
jacdelbutton.place(x=25, y=290)
#Get the output of Modified Query to get increased terms/ terms to be added to the query.
querymodifierbutton = customtkinter.CTkButton(master=root, text="Modify your query", command=query_modifier)
querymodifierbutton.pack()
querymodifierbutton.place(x=725, y=130)


#Dice Button to get the top 10 similar documents .
dicebutton = customtkinter.CTkButton(master=root, text="Get the dice values", command=topdice)
dicebutton.pack()
dicebutton.place(x=25, y=320)

# Close the tkinter module.
def on_closing():
    root.quit()
    root.destroy()

# Set the function to be called when the window is closed
root.protocol("WM_DELETE_WINDOW", on_closing)


root.mainloop()
# root.destroy()



Exception in Tkinter callback
Traceback (most recent call last):
  File "R:\ANACONDA\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "R:\ANACONDA\lib\site-packages\customtkinter\windows\widgets\ctk_button.py", line 553, in _clicked
    self._command()
  File "C:\Users\ABDULR~1\AppData\Local\Temp/ipykernel_3168/3886248279.py", line 124, in <lambda>
    delbutton = customtkinter.CTkButton(master=root, text="Clear the Output", command=lambda: [mydelete()] ,fg_color="red")
  File "C:\Users\ABDULR~1\AppData\Local\Temp/ipykernel_3168/3886248279.py", line 58, in mydelete
    if cosine_label is not None:
NameError: name 'cosine_label' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "R:\ANACONDA\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "R:\ANACONDA\lib\site-packages\customtkinter\windows\widgets\ctk_button.py", line 553, in _clicked
    self._command()
  File "C:\Users\ABDULR~1\

## Evaluation of our Search Retrieval


In [37]:
# Fetching the Med Line Corpus Dataset
#Getting an reference on how to read the data from here
#https://github.com/pragmalingu/experiments/blob/master/00_Data/MedlineCorpus.ipynb
PATH_TO_MED_TXT = 'MED.ALL'
PATH_TO_MED_QRY = 'MED.QRY'
PATH_TO_MED_REL = 'MED.REL'

In [38]:
# Creating two list for med document and query
ID_marker = re.compile('^\.I',re.MULTILINE)

def get_data(PATH_TO_FILE, marker):
    with open (PATH_TO_FILE,'r') as f:
        text = f.read()
        lines = re.split(marker,text)
        lines.pop(0)
    return lines

med_txt_list = get_data(PATH_TO_MED_TXT, ID_marker)
med_qry_list = get_data(PATH_TO_MED_QRY, ID_marker)



In [39]:
# hash Map for documents
cacm_documents={}
for i in range(1,len(med_txt_list)):
    cacm_documents[i]=med_txt_list[i-1].replace('\n','').replace('\t',"")

In [40]:
# hash map for queries
cacm_queries={}
for i in range(1,len(med_qry_list)):
    cacm_queries[i]=med_qry_list[i-1].replace('\n','').replace('\t',"")


In [41]:
# Creating a function to remove punctuations, stopwords and stem words
def preprocessing_documents1():
    df_cacm ={}
    # loop through the dictionary and process the values
    for key, value in cacm_documents.items():
        # Removing the punctuations, numbers and characters.    
        value=re.sub(r'≥|µ|\d+', '', value).translate(translator).lower().split()
        #removing stop words
        filtered_words = [word for word in value if word not in stopwords and len(word) > 1]
        #Stemming of Words
        stemming = snowball.SnowballStemmer('english')
        stemming_wwrds = [stemming.stem(x) for x in filtered_words]
        df_cacm[key] = list(stemming_wwrds)

    return df_cacm

In [42]:
df_cacm=preprocessing_documents1()

In [43]:
# Creating a function to remove punctuations, stopwords and stem words
def preprocessing_documents2():
    df_cacmqueries ={}
    # loop through the dictionary and process the values
    for key, value in cacm_queries.items():

        value=re.sub(r'≥|µ|\d+', '', value).translate(translator).lower().split()

        filtered_words = [word for word in value if word not in stopwords and len(word) > 1]
        stemming = snowball.SnowballStemmer('english')
        stemming_wwrds = [stemming.stem(x) for x in filtered_words]
        df_cacmqueries[key] = list(stemming_wwrds)

    return df_cacmqueries

In [44]:
df_cacmqueries=preprocessing_documents2()

In [45]:
# creating a second dictionary to store the token and the list of documents it is present and the number of occurences
def doc_occurences1():
    second_dict={}
    for key,value in  df_cacm.items():
        for i in range(len(value)): # running through each key and its respective value
            if value[i] in second_dict:  # checking whether the value is present or not 
                if key not in second_dict[value[i]].keys(): 
    # this condition is because when the loop goes to second document id it has to add multiple occurences of a particular word
                    second_dict[value[i]][key]=1
                else:
                    second_dict[value[i]][key]+=1 # else increase the count by 1
            else:           
                second_dict[value[i]]={key:1} # create a multi level dictionary to add the words , the document id and the ouccrence
    return second_dict

In [46]:
df_cacmoccur=doc_occurences1()

In [56]:
sorted_dict = {key: value for key, value in sorted(doc_occurences1().items())}

In [57]:
# Getting the IDF Values for the documents.
idf_dict={}
N = len(med_txt_list)

def compute_tfidf1(term, doc_id):
    tf = sorted_dict[term][doc_id]
    idf = math.log2(N / len(sorted_dict[term]))
    idf_dict[term]=math.log2(N / len(sorted_dict[term]))
    return tf * idf

weighted_index = {}
for term in sorted_dict:
    weighted_index[term] = {}
    for doc_id in sorted_dict[term]:
        weighted_index[term][doc_id] = compute_tfidf(term, doc_id)

In [58]:
# Pre-processing of the user query.
def userquery1(query):
    query_idf={}
    query=re.sub(r'≥|µ|\d+', '', query).translate(translator).lower().split()

    filtered_words = [word for word in query if word not in stopwords and len(word) > 1]

    
    stemming = snowball.SnowballStemmer('english')
    stemming_wwrds = [stemming.stem(x) for x in filtered_words]
    for i in stemming_wwrds:
        if i in idf_dict.keys():
            query_idf[i]=idf_dict[i]
        else:
             pass
    return query_idf

In [59]:
# Document Lenght for all the similarities for the MEdline corpus
def doclength1(similarity):
    doc_length={}
    if similarity.lower()=='cosine':
        #for Cosine Similarity
        for key, value in sorted_dict.items():
            for document in value.keys():
                if document not in doc_length:
                    doc_length[document]=(sorted_dict[key][document]*idf_dict[key])**2
                else:
                    doc_length[document]+=(sorted_dict[key][document]*idf_dict[key])**2
        for key in doc_length:
            doc_length[key] = math.sqrt(doc_length[key])
    else:
        #for jaccard and dice
        for key, value in sorted_dict.items():
            for document in value.keys():
                if document not in doc_length:
                    doc_length[document]=(sorted_dict[key][document]*idf_dict[key])**2
                else:
                    doc_length[document]+=(sorted_dict[key][document]*idf_dict[key])**2
        for key in doc_length:
            doc_length[key] = np.sum(doc_length[key])
        
    return doc_length


In [60]:
# Getting the query length for the query and respective similarity
def querieslength1(q,similarity):
    sum_queries_length=0
    if similarity.lower()=='cosine':
        for key in q:
            sum_queries_length+= (idf_dict[key])**2
        sum_queries_length=math.sqrt(sum_queries_length)
    elif similarity.lower()=='jaccard' or similarity.lower()=='dice':
        for key in q:
            sum_queries_length+= (idf_dict[key])**2
        sum_queries_length=np.sum(sum_queries_length)      
    return sum_queries_length


In [61]:
# Similar documents for the query and the document only for the Cosine Similarity
def similar_documents1(q):
    similarities=["Cosine"]
    dictionaries_list=[]
    document_ids= list(range(0,len(med_txt_list)))
    document_dict = {document: 0 for document in document_ids}
    query_length=querieslength1(q,'Cosine')
    doc_length=doclength1('Cosine')
    for term in q:
        for key in sorted_dict[term].keys():
            document_dict[key]+=(sorted_dict[term][key]*idf_dict[term])*(1*idf_dict[term])
    for key in document_dict:
        if key in doclength1('Cosine'):
            document_dict[key] = document_dict[key]/(query_length*doc_length[key])

    dictionaries_list.append(document_dict)
 
    diff_similarity_df = pd.DataFrame.from_dict(dictionaries_list )
    diff_similarity_df=diff_similarity_df.T
    diff_similarity_df.columns = similarities
    return diff_similarity_df


In [62]:
%%time
# get the top 10 retrieved documents using Cosine Similarity 
retrieved_doc={}
for key,values in cacm_queries.items():
#     print(key)
    expanded_query=word2vec_expand(values, num_words=5, threshold=0.5)
    q=userquery1(expanded_query)
    dftestsim=similar_documents1(q)
    retrieved_doc[key]=list(dftestsim.sort_values(by='Cosine' , ascending=False).reset_index().rename(columns={'index':'Document ID'})['Document ID'].iloc[0:10])
print(retrieved_doc)

{1: [72, 509, 168, 965, 500, 360, 138, 142, 181, 180], 2: [258, 712, 162, 289, 299, 187, 237, 96, 713, 192], 3: [70, 41, 62, 59, 71, 234, 160, 230, 282, 276], 4: [67, 407, 234, 952, 93, 113, 532, 713, 175, 953], 5: [8, 5, 11, 308, 329, 159, 658, 327, 6, 333], 6: [112, 243, 323, 253, 267, 116, 242, 238, 122, 321], 7: [121, 189, 261, 92, 247, 82, 392, 391, 114, 112], 8: [52, 435, 265, 60, 434, 426, 250, 264, 1026, 262], 9: [421, 413, 114, 194, 82, 56, 75, 97, 409, 268], 10: [572, 532, 534, 281, 77, 209, 543, 677, 702, 176], 11: [780, 92, 990, 122, 228, 445, 318, 446, 316, 225], 12: [19, 367, 368, 853, 20, 193, 371, 220, 744, 768], 13: [197, 198, 21, 196, 195, 199, 194, 481, 144, 147], 14: [455, 457, 23, 25, 368, 26, 467, 853, 461, 454], 15: [218, 361, 103, 219, 106, 355, 893, 353, 102, 357], 16: [202, 36, 99, 205, 98, 812, 486, 484, 1012, 253], 17: [41, 336, 38, 472, 128, 216, 342, 129, 379, 528], 18: [521, 528, 526, 45, 134, 49, 81, 548, 517, 525], 19: [847, 844, 555, 440, 32, 507, 441,

In [63]:
# relevant Documents.
med_rel_data = open(PATH_TO_MED_REL)
med_np = np.loadtxt(med_rel_data, dtype=int)

med_rel_rat = defaultdict(list)
for row in med_np:
    med_rel_rat[row[0]].append(row[2])


In [65]:
# Intersection of Relevant and Retrieved Documents
def find_matching_values(dict1, dict2):
    count = 0
    for key1, value1 in dict1.items():
        if key1 in dict2:
            for val1 in value1:
#                 print(val1 ,dict2[key1] )
                if val1 in dict2[key1]:
                    count += 1
    return count

In [66]:
# Intersection of Relevant and Retrieved Documents
inte_rel_ret = find_matching_values(retrieved_doc, med_rel_rat)
inte_rel_ret

174

In [67]:
# Number of retrieved documents 
count_retr = 0
for values in retrieved_doc.values():
    count_retr += len(values)
print(count_retr)

290


In [68]:
# Number of relevance documents
count_rel = 0
for values in med_rel_rat.values():
    count_rel += len(values)
print(count_rel)

696


In [69]:
# Calculate Precison and Recall

Precision = inte_rel_ret/count_retr

Recall = inte_rel_ret/count_rel

print(f'The precision for the test data set is {Precision} and the recall is {Recall}')

The precision for the test data set is 0.6 and the recall is 0.25
